# Free-Energy Decomposition of Salt Effects on the Solubilities of Small Molecules and the Role of Excluded-Volume Effects
Stefan Hervø-Hansen<sup>a,&#42;</sup>, Lin Daoyang<sup>a</sup>, Kento Kasahara<sup>a</sup>, and Nobuyuki Matubayasi<sup>a,&#42;</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## PART 2: Simulations 

## Import of Python Modules & Auxiliary Functions

In [ ]:
# Notebook dependent libs
import numpy as np
import matplotlib.pyplot as plt
import os, re, time
from shutil import copyfile
from distutils.spawn import find_executable
import parmed as pmd
from openmm import app
import openmm as mm
import mdtraj as md

# Simulation specific libs
# from openmm import app
# import openmm as mm
# import mdtraj as md

# Check for external programs
if None in [find_executable('packmol')]:
    print('WARNING: External program missing!')
    
# Physical constants & useful conversions
Na = 6.02214076e23 # Avogadro constant [mol]
liter_to_cubeangstrom = 1e27


homedir = !pwd
homedir = homedir[0]
workdir = '/home/users/eau/SI-Cavity-Formation'
print(homedir)

## Molecular Dynamics Simulations
### Simulation settings

In [ ]:
states = { # State of simulations, (outFreq is steps per frame)
          'vac': {'Nsteps':   1000000, 'OutFreq':  10}, #   1 nanoseconds,  100000 frames
          'sol': {'Nsteps':  10000000, 'OutFreq': 100}, #  20 nanoseconds,  100000 frames
          'ref': {'Nsteps':  25000000, 'OutFreq': 100}, #  50 nanoseconds,  250000 frames
          'rdf': {'Nsteps':  10000000, 'OutFreq': 100}, #  20 nanoseconds,  100000 frames

}

salts = {
    'NaF':  {'cation': 'Na', 'anion': 'F' },
    'NaCl': {'cation': 'Na', 'anion': 'Cl'},
    'NaI':  {'cation': 'Na', 'anion': 'I' },
    'KF':   {'cation': 'K',  'anion': 'F' },
    'KCl':  {'cation': 'K',  'anion': 'Cl'},
    'KI':   {'cation': 'K',  'anion': 'I' },
    'CsF':  {'cation': 'Cs', 'anion': 'F' },
    'CsCl': {'cation': 'Cs', 'anion': 'Cl'},
    'CsI':  {'cation': 'Cs', 'anion': 'I' },
}

solutes = {
    'methanol':   {'SMILES': 'CO',      'Category': 'n-alcohols', 'Resname': 'MOH'},
    'ethanol' :   {'SMILES': 'CCO',     'Category': 'n-alcohols', 'Resname': 'EOH'},
    'n-propanol': {'SMILES': 'CCCO',    'Category': 'n-alcohols', 'Resname': 'PHO'},
    'n-butanol' : {'SMILES': 'CCCCO',   'Category': 'n-alcohols', 'Resname': 'BOH'},
    'n-pentanol': {'SMILES': 'CCCCCO',  'Category': 'n-alcohols', 'Resname': 'POH'},
    'n-hexanol':  {'SMILES': 'CCCCCCO', 'Category': 'n-alcohols', 'Resname': 'HHO'},
    
    'methane':    {'SMILES': 'C',       'Category': 'n-alkanes',  'Resname': 'MTH'},
    'ethane' :    {'SMILES': 'CC',      'Category': 'n-alkanes',  'Resname': 'ETH'},
    'propane':    {'SMILES': 'CCC',     'Category': 'n-alkanes',  'Resname': 'PRP'},
    'n-butane' :  {'SMILES': 'CCCC',    'Category': 'n-alkanes',  'Resname': 'BUT'},
    'n-pentane':  {'SMILES': 'CCCCC',   'Category': 'n-alkanes',  'Resname': 'PEN'},
    'n-hexane':   {'SMILES': 'CCCCCC',  'Category': 'n-alkanes',  'Resname': 'HEX'},
}

concentrations = {
     0.00: {'N_solute':1, 'N_water':10000, 'N_cation':0,   'N_anion':0},
     0.50: {'N_solute':1, 'N_water':10000, 'N_cation':90,  'N_anion':90},
     1.00: {'N_solute':1, 'N_water':10000, 'N_cation':180, 'N_anion':180}
}

GENERATE_SOLUTES    = False  # Generate the structual ensemble of a solute in vacuum
GENERATE_REFERENCES = False  # Generate the reference conditions
GENERATE_SOLUTIONS  = False  # Generate the solution conditions
GENERATE_RDF        = False  # Generate simulations for RDF calculations

## Step 1. Generation of Vacuum Solute Configurations
### Construction of initial configurations and topology file

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output Initial.pdb

add_box_sides 1.0

structure {homedir}/PDB_files/{category}/{solute}.pdb
        number 1
        fixed 10. 10. 10. 0. 0. 0.
        centerofmass
end structure
"""


topology_file = """
; Include forcefield files
#include "{homedir}/Force_fields/{solute}.itp"

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               2               no              1            0.83333333

[ system ]
{solute} in vacuum

[ molecules ]
;molecule name   nr.
{solute_resname}             1
"""

if GENERATE_SOLUTES:
    for solute, solute_details in solutes.items():
        %cd -q $homedir/Simulations/$solute/Solute
          
        # Packmol input
        with open('packmol.in', 'w') as f:
            f.write(packmol_script.format(homedir=homedir, category=solute_details['Category'], solute=solute))
        f.close()
        !packmol < packmol.in
        
        # Topology input
        with open('Topology.top', 'w') as f:
            f.write(topology_file.format(homedir=workdir, solute=solute, solute_resname=solute_details['Resname']))
        f.close()
        
        print('Wrote initial configurations and topology files to'+os.getcwd())

### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('Initial.pdb')
top = app.GromacsTopFile('Topology.top', periodicBoxVectors=pdb.topology.getPeriodicBoxVectors())

# Creating system
print('Creating OpenMM System')
system = top.createSystem(nonbondedMethod=app.NoCutoff, constraints=app.HBonds,
                          rigidWater=True, removeCMMotion=True)
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 1.0*u.femtoseconds)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(top.topology, system, integrator, platform, properties)

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy()
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*1 fs = 0.5 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})
"""           

if GENERATE_SOLUTES:
    N_simulations=0
    for solute, solute_details in solutes.items():
        %cd -q $homedir/Simulations/$solute/Solute
        
        with open('openMM.py', 'w') as f:
            f.write(openmm_script.format(Nsteps=states['vac']['Nsteps'], outFreq=states['vac']['OutFreq']))
        f.close()
        print('Wrote run_openMM.py files to '+os.getcwd())
        N_simulations+=1

    print('Simulations about to be submitted: {}'.format(N_simulations))

### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
#PBS -l select=1:ncpus=1:mpiprocs=1:ompthreads=1:jobtype=gpu:ngpus=1
#PBS -l walltime=4:00:00
#PBS -N Solute
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""

if GENERATE_SOLUTES:     
    with open('Simulations/master_solute.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
        
        for solute in solutes:
            %cd -q $homedir/Simulations/$solute/Solute
            
            simpath = os.getcwd()
            simpath = simpath.split(sep='SI-Cavity-Formation')[1]
            simpath = workdir+simpath
            with open('submit.pbs', 'w') as f:
                f.write(submit_script.format(solute=solute, path=simpath))
            f.close()
            s = 'cd {}\njsub submit.pbs\n\n'.format(simpath)
            ff.write(s)
    ff.close()

## Step 2. Generation of Solvent Configurations - Reference System (λ = 0)
### Construction of initial configurations and topology file

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output Initial.pdb

add_box_sides 0.1

{saltfix}structure {homedir}/PDB_files/Ions/{cation}.pdb
{saltfix}        number {N_cation}
{saltfix}        inside cube 0. 0. 0. 67.
{saltfix}end structure
        
{saltfix}structure {homedir}/PDB_files/Ions/{anion}.pdb
{saltfix}        number {N_anion}
{saltfix}        inside cube 0. 0. 0. 67.
{saltfix}end structure

structure {homedir}/PDB_files/spce.pdb
        number {N_water}
        inside cube 0. 0. 0. 67.
end structure
"""


topology_file = """
; Include forcefield files
#include "{homedir}/Force_fields/ions.itp"
#include "{homedir}/Force_fields/spce.itp"

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               2               no              1            0.83333333

[ system ]
Salt water

[ molecules ]
;molecule name   nr.
{saltfix}{cation_resname}          {N_cation}
{saltfix}{anion_resname}          {N_anion}
SOL          {N_water}
"""

if GENERATE_REFERENCES:
    for i, (salt, salt_details) in enumerate(salts.items()):
        for conc, conc_details in concentrations.items():
            if conc == 0 and i == 0:
                %cd -q $homedir/Simulations/References/No_salt
            elif conc == 0 and i > 0:
                continue
            else:
                conc = '{0:.2f}'.format(conc)
                %cd -q $homedir/Simulations/References/$salt/$conc
                
            # Construction of initial configuration
            with open('packmol.in', 'w') as f:
                # Fix for no salt
                if float(conc) == 0:
                    saltfix='#'
                else:
                    saltfix=''
                f.write(packmol_script.format(homedir=homedir, saltfix=saltfix, N_water=conc_details['N_water'],
                                              cation=salt_details['cation'], anion=salt_details['anion'],
                                              N_cation=conc_details['N_cation'], N_anion=conc_details['N_anion']))
            f.close()
            !packmol < packmol.in
            
            # Construction of topology file
            with open('Topology.top', 'w') as f:
                if float(conc) == 0:
                    saltfix=';'
                else:
                    saltfix=''
                f.write(topology_file.format(homedir=workdir, saltfix=saltfix,
                                             cation_resname=salt_details['cation'].upper(),
                                             anion_resname=salt_details['anion'].upper(),
                                             N_water=conc_details['N_water'],
                                             N_cation=conc_details['N_cation'],
                                             N_anion=conc_details['N_anion']
                                             ))
            f.close()
            print('Wrote initial configurations and topology files to'+os.getcwd())

### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('Initial.pdb')
top = app.GromacsTopFile('Topology.top', periodicBoxVectors=pdb.topology.getPeriodicBoxVectors())

# Creating system
print('Creating OpenMM System')
system = top.createSystem(nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                          nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True, 
                          removeCMMotion=True)
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(top.topology, system, integrator, platform, properties)

# Create a dictionary to store all forces
forces = {{sim.system.getForce(index).__class__.__name__: sim.system.getForce(index) for index in range(sim.system.getNumForces())}}

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy()
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*2 fs = 1.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
"""           

if GENERATE_REFERENCES:
    N_simulations = 0
    for i, salt in enumerate(salts):
        for conc in concentrations:
            if conc == 0 and i == 0:
                %cd -q $homedir/Simulations/References/No_salt
            elif conc == 0 and i > 0:
                continue
            else:
                conc = '{0:.2f}'.format(conc)
                %cd -q $homedir/Simulations/References/$salt/$conc
        
            with open('openMM.py', 'w') as f:
                f.write(openmm_script.format(Nsteps=states['ref']['Nsteps'], outFreq=states['ref']['OutFreq']))
            f.close()
            print('Wrote run_openMM.py files to '+os.getcwd())
            N_simulations+=1

print('Simulations about to be submitted: {}'.format(N_simulations))

### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
#PBS -l select=1:ncpus=1:mpiprocs=1:ompthreads=1:jobtype=gpu:ngpus=1
#PBS -l walltime=4:00:00
#PBS -N Reference
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""

if GENERATE_REFERENCES:
    with open('Simulations/master_reference.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
        for i, salt in enumerate(salts):
            for conc in concentrations:
                if conc == 0 and i == 0:
                    %cd -q $homedir/Simulations/References/No_salt
                elif conc == 0 and i > 0:
                    continue
                else:
                    conc = '{0:.2f}'.format(conc)
                    %cd -q $homedir/Simulations/References/$salt/$conc
                
                simpath = os.getcwd()
                simpath = simpath.split(sep='SI-Cavity-Formation')[1]
                simpath = workdir+simpath
                with open('submit.pbs', 'w') as f:
                    f.write(submit_script.format(path=simpath))
                f.close()
                s = 'cd {}\njsub submit.pbs\n\n'.format(simpath)
                ff.write(s)
        f.close()

## Step 3. Generation of Solvent Configurations in Presence of Solute -<br> Solution System (λ = 1)
### Construction of initial configurations and topology file

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output Initial.pdb

add_box_sides 0.1

structure {homedir}/PDB_files/{category}/{solute}.pdb
        number 1
        fixed 33.5 33.5 33.5 0. 0. 0.
        centerofmass
end structure

{saltfix}structure {homedir}/PDB_files/Ions/{cation}.pdb
{saltfix}        number {N_cation}
{saltfix}        inside cube 0. 0. 0. 67.
{saltfix}end structure
        
{saltfix}structure {homedir}/PDB_files/Ions/{anion}.pdb
{saltfix}        number {N_anion}
{saltfix}        inside cube 0. 0. 0. 67.
{saltfix}end structure

structure {homedir}/PDB_files/spce.pdb
        number {N_water}
        inside cube 0. 0. 0. 67
end structure
"""


topology_file = """
; Include forcefield files
#include "{homedir}/Force_fields/ions.itp"
#include "{homedir}/Force_fields/spce.itp"
#include "{homedir}/Force_fields/{solute}.itp"

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               2               no              1            0.83333333

[ system ]
{solute} in salt water

[ molecules ]
;molecule name   nr.
{solute_resname}          1
{saltfix}{cation_resname}          {N_cation}
{saltfix}{anion_resname}          {N_anion}
SOL          {N_water}
"""

if GENERATE_SOLUTIONS:
    water_done = False
    for solute, solute_details in solutes.items():
        for salt, salt_details in salts.items():
            for conc, conc_details in concentrations.items():
                if conc == 0 and water_done == False:
                    %cd -q $homedir/Simulations/$solute/No_salt
                    water_done = True
                elif conc == 0 and water_done == True:
                    continue
                else:
                    conc = '{0:.2f}'.format(conc)
                    %cd -q $homedir/Simulations/$solute/$salt/$conc
                
                # Construction of initial configuration
                with open('packmol.in', 'w') as f:
                    # Fix for no salt
                    if float(conc) == 0:
                        saltfix='#'
                    else:
                        saltfix=''
                    f.write(packmol_script.format(homedir=homedir, saltfix=saltfix,
                                                  category=solute_details['Category'], solute=solute, N_water=conc_details['N_water'],
                                                  cation=salt_details['cation'], N_cation=conc_details['N_cation'], 
                                                  anion=salt_details['anion'], N_anion=conc_details['N_anion']))
                f.close()
                !packmol < packmol.in
                
                # Construction of topology file
                with open('Topology.top', 'w') as f:
                    if float(conc) == 0:
                        saltfix=';'
                    else:
                        saltfix=''
                    f.write(topology_file.format(homedir=homedir, saltfix=saltfix,
                                                 solute=solute, solute_resname=solute_details['Resname'],
                                                 cation_resname=salt_details['cation'].upper(),
                                                 anion_resname=salt_details['anion'].upper(),
                                                 N_water=conc_details['N_water'],
                                                 N_cation=conc_details['N_cation'],
                                                 N_anion=conc_details['N_anion']
                                                 ))
                f.close()
                
                mol = pmd.load_file('Topology.top', xyz='Initial.pdb')
                mol.save('Topology_stand_alone.top', overwrite=True)
                
                print('Wrote initial configurations and topology files to '+os.getcwd())
        water_done = False

### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('Initial.pdb')
top = app.GromacsTopFile('Topology_stand_alone.top', periodicBoxVectors=pdb.topology.getPeriodicBoxVectors())

# Creating system
print('Creating OpenMM System')
system = top.createSystem(nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                          nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True, 
                          removeCMMotion=True)
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(top.topology, system, integrator, platform, properties)

# Create a dictionary to store all forces
forces = {{sim.system.getForce(index).__class__.__name__: sim.system.getForce(index) for index in range(sim.system.getNumForces())}}

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy()
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*2 fs = 1.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
"""           

N_simulations = 0
if GENERATE_SOLUTIONS:
    water_done = False
    for solute, solute_details in solutes.items():
        for salt, salt_details  in salts.items():
            for conc, conc_details in concentrations.items():
                if conc == 0 and water_done == False:
                    %cd -q $homedir/Simulations/$solute/No_salt
                    water_done = True
                elif conc == 0 and water_done == True:
                    continue
                else:
                    conc = '{0:.2f}'.format(conc)
                    %cd -q $homedir/Simulations/$solute/$salt/$conc
        
                with open('openMM.py', 'w') as f:
                    f.write(openmm_script.format(Nsteps=states['sol']['Nsteps'], outFreq=states['sol']['OutFreq']))
                f.close()
                print('Wrote run_openMM.py files to '+os.getcwd())
                N_simulations+=1
        water_done = False

print('Simulations about to be submitted: {}'.format(N_simulations))

### Submit script

In [ ]:
%cd -q $homedir

submit_script="""#!/bin/bash
#PBS -l select=1:ncpus=1:mpiprocs=1:ompthreads=1:jobtype=gpu:ngpus=1
#PBS -l walltime=4:00:00
#PBS -N Solution
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""

if GENERATE_SOLUTIONS:
    with open('Simulations/master_solution.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
        water_done = False
        for solute, solute_details in solutes.items():
            for salt, salt_details  in salts.items():
                for conc, conc_details in concentrations.items():
                    if conc == 0 and water_done == False:
                        %cd -q $homedir/Simulations/$solute/No_salt
                        water_done = True
                    elif conc == 0 and water_done == True:
                        continue
                    else:
                        conc = '{0:.2f}'.format(conc)
                        %cd -q $homedir/Simulations/$solute/$salt/$conc
                        
                    simpath = os.getcwd()
                    simpath = simpath.split(sep='SI-Cavity-Formation')[1]
                    simpath = workdir+simpath
            
                    with open('submit.pbs', 'w') as f:
                        f.write(submit_script.format(path=simpath))
                    f.close()
                    s = 'cd {}\njsub submit.pbs\n\n'.format(simpath)
                    ff.write(s)
            water_done = False
    f.close()

## EXTRA: RDF Simulations
### Conditions

In [ ]:
salts = {
    'KF':   {'cation': 'K',  'anion': 'F' },
    'KCl':  {'cation': 'K',  'anion': 'Cl'},
    'KI':   {'cation': 'K',  'anion': 'I' },
}

solutes = {
    'n-hexane':   {'SMILES': 'CCCCCC',  'Category': 'n-alkanes',  'Resname': 'HEX'},
}

concentrations = {
     1.00: {'N_solute':1, 'N_water':10000, 'N_cation':180, 'N_anion':180}
}

Nreplica = 50

### Construction of initial configurations and topology file

In [ ]:
%cd -q $homedir

packmol_script="""
tolerance 2.0
filetype pdb
output Initial.pdb

add_box_sides 0.1

structure {homedir}/PDB_files/{category}/{solute}.pdb
        number 1
        fixed 33.5 33.5 33.5 0. 0. 0.
        centerofmass
end structure

structure {homedir}/PDB_files/Ions/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 67.
end structure

structure {homedir}/PDB_files/Ions/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 67.
end structure

structure {homedir}/PDB_files/spce.pdb
        number {N_water}
        inside cube 0. 0. 0. 67
end structure
"""


topology_file = """
; Include forcefield files
#include "{homedir}/Force_fields/ions.itp"
#include "{homedir}/Force_fields/spce.itp"
#include "{homedir}/Force_fields/{solute}.itp"

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               2               no              1            0.83333333

[ system ]
{solute} in salt water

[ molecules ]
;molecule name   nr.
{solute_resname}          1
{cation_resname}          {N_cation}
{anion_resname}          {N_anion}
SOL          {N_water}
"""

if GENERATE_RDF:
    for solute, solute_details in solutes.items():
        for salt, salt_details in salts.items():
            for conc, conc_details in concentrations.items():
                conc = '{0:.2f}_rdf'.format(conc)
                for replica in range(Nreplica):
                    %cd -q $homedir/Simulations/$solute/$salt/$conc/$replica
                
                    # Construction of initial configuration
                    with open('packmol.in', 'w') as f:
                        f.write(packmol_script.format(homedir=homedir, 
                                                      category=solute_details['Category'], solute=solute, N_water=conc_details['N_water'],
                                                      cation=salt_details['cation'], N_cation=conc_details['N_cation'], 
                                                      anion=salt_details['anion'], N_anion=conc_details['N_anion']))
                    f.close()
                    !packmol < packmol.in
                    
                    # Construction of topology file
                    with open('Topology.top', 'w') as f:
                        f.write(topology_file.format(homedir=homedir,
                                                     solute=solute, solute_resname=solute_details['Resname'],
                                                     cation_resname=salt_details['cation'].upper(),
                                                     anion_resname=salt_details['anion'].upper(),
                                                     N_water=conc_details['N_water'],
                                                     N_cation=conc_details['N_cation'],
                                                     N_anion=conc_details['N_anion']
                                                     ))
                    f.close()
                    
                    mol = pmd.load_file('Topology.top', xyz='Initial.pdb')
                    mol.save('Topology_stand_alone.top', overwrite=True)
                    
                    print('Wrote initial configurations and topology files to '+os.getcwd())

### Molecular dynamics setup using OpenMM

In [ ]:
%cd -q $homedir

openmm_script="""# Imports
import sys
import os
import openmm as mm
from openmm import app
from openmm import unit as u
from mdtraj.reporters import XTCReporter

print('Loading initial configuration and toplogy')
pdb = app.PDBFile('Initial.pdb')
top = app.GromacsTopFile('Topology_stand_alone.top', periodicBoxVectors=pdb.topology.getPeriodicBoxVectors())

# Creating system
print('Creating OpenMM System')
system = top.createSystem(nonbondedMethod=app.PME, ewaldErrorTolerance=0.0005, switchDistance=1*u.nanometer,
                          nonbondedCutoff=1.2*u.nanometers, constraints=app.HBonds, rigidWater=True, 
                          removeCMMotion=True)
        
# Temperature-coupling by leap frog (BAOAB) Langevin integrator (NVT)
integrator = mm.LangevinMiddleIntegrator(298.15*u.kelvin, 1.0/u.picoseconds, 2.0*u.femtoseconds)

# Pressure-coupling by a Monte Carlo Barostat (NPT)
system.addForce(mm.MonteCarloBarostat(1*u.bar, 298.15*u.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {{'CudaPrecision': 'mixed', 'CudaDeviceIndex': '0'}}

# Create the Simulation object
sim = app.Simulation(top.topology, system, integrator, platform, properties)

# Create a dictionary to store all forces
forces = {{sim.system.getForce(index).__class__.__name__: sim.system.getForce(index) for index in range(sim.system.getNumForces())}}

# Set the particle positions
sim.context.setPositions(pdb.positions)

# Minimize the energy
print('Minimizing energy')
sim.minimizeEnergy()
    
# Draw initial MB velocities
sim.context.setVelocitiesToTemperature(298.15*u.kelvin)

# Equlibrate simulation
print('Equilibrating...')
sim.step(500000)  # 500000*2 fs = 1.0 ns

# Set up the reporters
sim.reporters.append(app.StateDataReporter('output.dat', {outFreq}, totalSteps={Nsteps}+500000,
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True, volume=True, density=True,
    remainingTime=True, speed=True, separator='\t'))

# Set up trajectory reporter
sim.reporters.append(XTCReporter('trajectory.xtc', reportInterval={outFreq}, append=False))

# Run dynamics
print('Running dynamics! (NPT)')
sim.step({Nsteps})

# Print PME information
print('''
PARTICLE MESH EWALD PARAMETERS
Separation parameter: {{}}
Number of grid points along the X axis: {{}}
Number of grid points along the Y axis: {{}}
Number of grid points along the Z axis: {{}}
'''.format(*forces['NonbondedForce'].getPMEParametersInContext(sim.context)))
"""           

N_simulations = 0
if GENERATE_RDF:
    for solute, solute_details in solutes.items():
        for salt, salt_details in salts.items():
            for conc, conc_details in concentrations.items():
                conc = '{0:.2f}_rdf'.format(conc)
                for replica in range(Nreplica):
                    %cd -q $homedir/Simulations/$solute/$salt/$conc/$replica
        
                    with open('openMM.py', 'w') as f:
                        f.write(openmm_script.format(Nsteps=states['sol']['Nsteps'], outFreq=states['sol']['OutFreq']))
                    f.close()
                    print('Wrote run_openMM.py files to '+os.getcwd())
                    N_simulations+=1

print('Simulations about to be submitted: {}'.format(N_simulations))

### Submit script

In [ ]:
submit_script="""#!/bin/bash
#PBS -q F1accs
#PBS -N MD_RDF
#PBS -l select=1:ncpus=16:mpiprocs=16:ompthreads=1
#PBS -l walltime=2:00:00
#PBS -e run.err
#PBS -o run.out

source ~/.bashrc
source ~/.bash_profile

module load nvhpc-nompi/22.2_cuda11.6

cd {path}

python openMM.py
"""

master_submit_script="""#!/bin/bash
read -p "Will submit {N_simulations} MD simulations. Do you want to proceed? (yes/no) " yn

case $yn in 
    yes ) echo submitting...;;
    no ) echo exiting...;
         exit;;
    * ) echo invalid response;
        exit 1;;
esac

"""

%cd -q $homedir
workdir = '/home/m0053/m005302/work/SI-Cavity-Formation'

if GENERATE_RDF:
    with open('Simulations/master_RDF.sh', 'w') as ff:
        ff.write(master_submit_script.format(N_simulations=N_simulations))
        for solute, solute_details in solutes.items():
            for salt, salt_details in salts.items():
                for conc, conc_details in concentrations.items():
                    conc = '{0:.2f}_rdf'.format(conc)
                    for replica in range(Nreplica):
                        %cd -q $homedir/Simulations/$solute/$salt/$conc/$replica
                
                        simpath = os.getcwd()
                        simpath = simpath.split(sep='SI-Cavity-Formation')[1]
                        simpath = workdir+simpath
                        with open('submit_md.pbs', 'w') as f:
                            f.write(submit_script.format(path=simpath))
                        f.close()
                        s = 'cd {}\nqsub submit_md.pbs\n\n'.format(simpath)
                        ff.write(s)
    ff.close()